The basic idea is that neural networks learn to detect simple patterns in the top layer,
and then the next layer uses that information to look for slightly more complex patterns 
and so on, down through all the convolutional layers. 
But the final layer of the neural network is a densely connected layer that uses the information from the convolutional layers to decide which object is in the image. 
With transfer learning, we're gonna start with a neural network that's already been trained to recognize objects from a large dataset like ImageNet. 
To reuse this neural network with new data, 
we can simply slice off the last layer.

We'll keep all the layers that detect patterns, but remove the part that maps those patterns to specific objects.
We'll call this pre-trained neural network a feature extractor because we're using it to extract training features from images. 
Next, we'll create a new neural network to replace the last layer in the original network. 
This is the only part that we'll have to train ourselves. When we build our new image recognition system, we'll pass our new training images through the feature extractor and save the results for each training image to a file. 
Then, we'll use those extracted features to train the new neural network.


Transfer learning helps you to reduce the time spend in building a new neural network from sxratch.
Transfer learning is also very useful if you have small amount of training data.

In [3]:
from pathlib import Path
import numpy as np
import joblib
from keras.preprocessing import image
from keras.applications import vgg16

Using TensorFlow backend.


In [4]:
# Path to folders with training data
dog_path = Path("training_data") / "dogs"
not_dog_path = Path("training_data") / "not_dogs"


images = []
labels = []

In [5]:
#The first step is to load all images that are not dogs

# Load all the not-dog images
for img in not_dog_path.glob("*.png"):
    # Load the image from disk
    img = image.load_img(img)

    # Convert the image to a numpy array
    image_array = image.img_to_array(img)

    # Add the image to the list of images
    images.append(image_array)

    # For each 'not dog' image, the expected value should be 0
    labels.append(0)

In [6]:
# Load all the dog images. The only difference between loading the images that are not dogs
# and loading the ones that are dogs is that the number 1 is appended to the labels or picturre of the dog
for img in dog_path.glob("*.png"):
    # Load the image from disk
    img = image.load_img(img)

    # Convert the image to a numpy array
    image_array = image.img_to_array(img)

    # Add the image to the list of images
    images.append(image_array)

    # For each 'dog' image, the expected value should be 1
    labels.append(1)

In [7]:
# Create a single numpy array with all the images we loaded
x_train = np.array(images)

In [8]:
# Also convert the labels to a numpy array
y_train = np.array(labels)

In [9]:
# Normalize image data to 0-to-1 range
#When the image is processed, we need to remember which images were dogs and which ones were not dogs
#If the images are dog, we add 1 and wheen the images are not dogs, we add 0
x_train = vgg16.preprocess_input(x_train)

In [11]:
# Load a pre-trained neural network to use as a feature extractor
#The new neral network is created here by uding the VGG16 and passing the weights, input_shape and include_tops
#The top is the last layer of the neural network so include_top=False means that we want to exclude the last layer of the neural network 
#
pretrained_nn = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

58892288/58889256 [==============================] - 141s 2us/step


In [12]:
# Extract features for each image (all in one pass)
#Let's feed all of our images into the neural network and capture the results
features_x = pretrained_nn.predict(x_train)


In [13]:
# Save the array of extracted features to a file
#The library joblib helps with saving these data 
joblib.dump(features_x, "x_train.dat")

['x_train.dat']

In [14]:
# Save the matching array of expected values to a file
joblib.dump(y_train, "y_train.dat")

['y_train.dat']

In [ ]:
#These extracted features will be used to train a new neural network